## Importing Packages

In [ ]:
import requests 
import pandas as pd 
import os
from tqdm import tqdm
from io import StringIO
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup
import re 
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from geopy.geocoders import Nominatim
import transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

## Data Cleaning

In [295]:
trumpdf = pd.read_csv('hashtag_donaldtrump.csv', engine='python')

In [296]:
bidendf = pd.read_csv('hashtag_joebiden.csv', engine='python')

In [297]:
trumpdf = trumpdf[trumpdf.state.notnull()]
bidendf = bidendf[bidendf.state.notnull()]

In [298]:
final_df = pd.concat([trumpdf, bidendf], ignore_index=True)

In [299]:
final_df.columns

Index(['created_at', 'tweet_id', 'tweet', 'likes', 'retweet_count', 'source',
       'user_id', 'user_name', 'user_screen_name', 'user_description',
       'user_join_date', 'user_followers_count', 'user_location', 'lat',
       'long', 'city', 'country', 'continent', 'state', 'state_code',
       'collected_at'],
      dtype='object')

In [300]:
usersdf = pd.read_csv('users.csv')

/var/folders/0z/98hl6yf107q35dj2790zq94h0000gn/T/ipykernel_51170/3621589000.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  usersdf = pd.read_csv('users.csv')


In [301]:
usersdf.rename(columns={'user_id':'user_handle'}, inplace=True)

In [302]:
politiciandf = usersdf[usersdf['is_politician'] == 'Yes']

In [303]:
politiciandf = politiciandf.sort_values(by='user_handle')

In [304]:
politiciandf['user_id'] = range(len(politiciandf))

In [305]:
politiciandf

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
430,AGBecerra,NaN,California,NaN,United States of America,Yes,Democrat,0
343,AlanGrayson,NaN,Florida,NaN,United States of America,Yes,Democrat,1
314,AnthonyBrownMD4,NaN,Maryland,NaN,United States of America,Yes,Democrat,2
171,AustinScottGA08,NaN,Georgia,NaN,United States of America,Yes,Republican,3
381,BennieGThompson,NaN,Mississippi,NaN,United States of America,Yes,Democrat,4
...,...,...,...,...,...,...,...,...
2,reppittenger,NaN,North Carolina,NaN,United States of America,Yes,Republican,428
328,repsandylevin,NaN,Michigan,NaN,United States of America,Yes,Democrat,429
427,rosadelauro,NaN,Connecticut,NaN,United States of America,Yes,Democrat,430
357,sethmoulton,NaN,Massachusetts,NaN,United States of America,Yes,Democrat,431


In [306]:
nonpoliticiandf =  usersdf[usersdf['is_politician'] != 'Yes']

In [307]:
nonpoliticiandf.sort_values(by='user_handle')

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation
281166,0000000ef,0.0,NaN,NaN,NaN,No,NaN
281165,0000StingRay,451.0,NaN,NaN,NaN,No,NaN
281164,00010001b,41.0,NaN,NaN,NaN,No,NaN
281163,00010101010010O,0.0,NaN,NaN,NaN,No,NaN
281162,00010111_,782.0,NaN,NaN,NaN,No,NaN
...,...,...,...,...,...,...,...
140793,zzzbeeme,111.0,NaN,NaN,NaN,No,NaN
140792,zzzein26,2107.0,NaN,NaN,NaN,No,NaN
140791,zzzubair,416.0,NaN,NaN,India,No,NaN
140790,zzzz_accordd,356.0,NaN,NaN,NaN,No,NaN


In [342]:
len(nonpoliticiandf.user_id.unique())

129381

In [308]:
final_df = final_df[['tweet_id', 'user_id', 'user_screen_name', 'created_at', 'tweet', 'likes', 'retweet_count']].copy()
final_df.rename(columns={'user_screen_name' : 'user_handle'}, inplace=True)

In [309]:
nonpoliticiandf = pd.merge(nonpoliticiandf, final_df[['user_handle', 'user_id']], on='user_handle', how='left')

In [310]:
nonpoliticiandf = nonpoliticiandf.drop_duplicates()

In [311]:
usersdf2 = pd.concat([politiciandf, nonpoliticiandf])

In [312]:
usersdf2[usersdf2['is_politician'] != 'Yes']

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
0,dovalraw,102.0,NaN,NaN,NaN,No,NaN,NaN
1,douknowhoyouar3,160.0,NaN,NaN,NaN,No,NaN,NaN
2,douwecdevries,95.0,Gelderland,Apeldoorn,The Netherlands,No,NaN,2190451560.0
5,doutv_,57.0,NaN,NaN,NaN,No,NaN,NaN
6,doungs,267.0,Quebec,Montreal,Canada,No,NaN,29478747.0
...,...,...,...,...,...,...,...,...
852451,MatthewClary8,4.0,NaN,NaN,NaN,No,NaN,NaN
852452,MatthewCicanese,483.0,District of Columbia,Washington,United States of America,No,NaN,2418660171.0
852453,MatthewChattle,781.0,England,London,United Kingdom,No,NaN,250564463.0
852457,MatthewCarl76,74.0,NaN,NaN,NaN,No,NaN,NaN


In [313]:
usersdf2[usersdf2['is_politician'] == 'Yes']

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
430,AGBecerra,NaN,California,NaN,United States of America,Yes,Democrat,0
343,AlanGrayson,NaN,Florida,NaN,United States of America,Yes,Democrat,1
314,AnthonyBrownMD4,NaN,Maryland,NaN,United States of America,Yes,Democrat,2
171,AustinScottGA08,NaN,Georgia,NaN,United States of America,Yes,Republican,3
381,BennieGThompson,NaN,Mississippi,NaN,United States of America,Yes,Democrat,4
...,...,...,...,...,...,...,...,...
2,reppittenger,NaN,North Carolina,NaN,United States of America,Yes,Republican,428
328,repsandylevin,NaN,Michigan,NaN,United States of America,Yes,Democrat,429
427,rosadelauro,NaN,Connecticut,NaN,United States of America,Yes,Democrat,430
357,sethmoulton,NaN,Massachusetts,NaN,United States of America,Yes,Democrat,431


In [314]:
usersdf2.sort_values(by=['user_handle'])

,user_handle,user_followers_count,state,city,country,is_politician,political_affiliation,user_id
560751,0000000ef,0.0,NaN,NaN,NaN,No,NaN,NaN
560750,0000StingRay,451.0,NaN,NaN,NaN,No,NaN,NaN
560749,00010001b,41.0,NaN,NaN,NaN,No,NaN,NaN
560748,00010101010010O,0.0,NaN,NaN,NaN,No,NaN,NaN
560747,00010111_,782.0,NaN,NaN,NaN,No,NaN,NaN
...,...,...,...,...,...,...,...,...
277369,zzzbeeme,111.0,NaN,NaN,NaN,No,NaN,NaN
277368,zzzein26,2107.0,NaN,NaN,NaN,No,NaN,NaN
277367,zzzubair,416.0,NaN,NaN,India,No,NaN,NaN
277366,zzzz_accordd,356.0,NaN,NaN,NaN,No,NaN,NaN


In [315]:
final_df.head()

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0


In [316]:
politiciantweets = pd.read_csv('ExtractedTweets.csv')

In [317]:
politiciantweets.rename(columns={'Handle' : 'user_handle', 'Tweet' : 'tweet'}, inplace=True)
politiciantweets.head()

,Party,user_handle,tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...


In [318]:
politiciantweets.drop(columns=['Party'], inplace=True)

In [319]:
politiciantweets['tweet_id'] = None
politiciantweets['created_at'] = None
politiciantweets['likes'] = None
politiciantweets['retweet_count'] = None

In [320]:
politiciandf2 = politiciandf.copy()
politiciantweets = pd.merge(politiciantweets, politiciandf2[['user_handle', 'user_id']], on='user_handle', how='left')

In [321]:
politiciantweets

,user_handle,tweet,tweet_id,created_at,likes,retweet_count,user_id
0,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",None,None,None,None,137
1,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,None,None,None,None,137
2,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,None,None,None,None,137
3,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,None,None,None,None,137
4,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,None,None,None,None,137
...,...,...,...,...,...,...,...
86455,RepTomPrice,Check out my op-ed on need for End Executive O...,None,None,None,None,342
86456,RepTomPrice,"Yesterday, Betty &amp; I had a great time lear...",None,None,None,None,342
86457,RepTomPrice,We are forever grateful for the service and sa...,None,None,None,None,342
86458,RepTomPrice,Happy first day of school @CobbSchools! #CobbB...,None,None,None,None,342


In [265]:
final_df.drop(columns=['continent', 'country', 'state', 'city', 'state_code'], inplace=True)

In [322]:
# Improved function to check if a value can be converted to float
def can_convert_to_float(value):
    try:
        float(value)
        return True
    except (TypeError, ValueError):
        return False

# Apply the function to determine if conversion to float is possible
final_df['can_convert'] = final_df['likes'].apply(can_convert_to_float)

# Display rows with problematic 'likes' values
erroneous_rows = final_df[~final_df['can_convert']]
print("Erroneous rows:")
print(erroneous_rows)

# Remove rows where 'likes' cannot be converted to float
final_df = final_df[final_df['can_convert']].copy()

# Drop the 'can_convert' column as it's no longer needed
final_df.drop(columns=['can_convert'], inplace=True)

Erroneous rows:
       tweet_id          user_id          user_handle  \
76696       1.0      thetrump411  2017-02-24 05:29:46   
88151       0.0      thetrump411  2017-02-24 05:29:46   
124794      0.0         KULMusic  2020-09-21 10:24:47   
179976      0.0        peraforex  2020-07-14 13:19:26   
199106      1.0  krdigitalmakers  2020-10-31 17:17:25   
276607      0.0       KennyWon22  2019-08-22 18:44:54   
299886      0.0  DavidMoufarrege  2010-03-24 14:48:24   
393474      0.0       bursacom16  2011-09-20 12:43:15   
444599      0.0        peraforex  2020-07-14 13:19:26   
553104      0.0  DavidMoufarrege  2010-03-24 14:48:24   

                                               created_at tweet  \
76696   #trumpquotes #trumptweets #Trumpmeme. https://...   0.0   
88151   #trumpquotes #trumptweets #Trumpmeme https://t...   0.0   
124794  #partyatthewhitehouse #kul #icelandicmusic #ic...   0.0   
179976  Fırsatı Kaçırmayın ! #forex #abdseçim #joebide...   0.0   
199106  Follow  &gt;&

In [323]:
final_df = pd.concat([final_df, politiciantweets])

/var/folders/0z/98hl6yf107q35dj2790zq94h0000gn/T/ipykernel_51170/1085536531.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, politiciantweets])


In [274]:
erroneous_rows.head(10)

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,can_convert
6020,2009-07-14 12:06:26,NaN,NaN,Saved by Grace!,688.0,ExpectTheHolySpirit,NaN,False
6024,2009-07-14 12:06:26,NaN,NaN,Saved by Grace!,688.0,ExpectTheHolySpirit,NaN,False
6073,2009-07-14 12:06:26,NaN,NaN,Saved by Grace!,688.0,ExpectTheHolySpirit,NaN,False
21869,2013-04-15 01:42:25,NaN,NaN,Merkelgegner/Trumpster/Gedankenverbrecher. Lau...,4552.0,Failed State Germany,NaN,False
28327,2009-07-14 12:06:26,NaN,NaN,Saved by Grace!,688.0,ExpectTheHolySpirit,NaN,False
28331,2009-07-14 12:06:26,NaN,NaN,Saved by Grace!,688.0,ExpectTheHolySpirit,NaN,False
32421,2014-02-05 16:36:46,Budapest,Europe,ARD / Hörfunk,734.0,"Budapest, Hungary",47.498382,False
48308,2009-07-14 12:06:26,NaN,NaN,Saved by Grace!,688.0,ExpectTheHolySpirit,NaN,False
60530,2009-07-14 12:06:26,NaN,NaN,Saved by Grace!,688.0,ExpectTheHolySpirit,NaN,False
65444,2011-03-06 17:44:19,Rouen,Europe,à ne suivre que 200 comptes je m'astreins.,286.0,Rouen,49.440459,False


In [329]:
final_df

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0
...,...,...,...,...,...,...,...
86455,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN
86456,None,342,RepTomPrice,None,"Yesterday, Betty &amp; I had a great time lear...",None,NaN
86457,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN
86458,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN


## NLP

In [325]:
# Setting up a CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tweet_matrix = vectorizer.fit_transform(final_df['tweet'])

# Vocabulary of words
vocab = vectorizer.get_feature_names_out()

In [326]:
# Number of topics
n_topics = 20
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [327]:
# Setting up NMF
nmf = NMF(n_components=n_topics, random_state=1,  l1_ratio=.5, init='nndsvd')
nmf_topics = nmf.fit_transform(tweet_matrix)




In [328]:
# Display the topics found by NMF
display_topics(nmf, vocab, 5)

Topic 0:
trump donald maga trump2020 supporters
Topic 1:
https today new rt live
Topic 2:
biden bidenharris2020 elections2020 harris votes
Topic 3:
joebiden kamalaharris bidenharris2020 congratulations joebidenkamalaharris2020
Topic 4:
que en el los se
Topic 5:
amp rt care pa gop
Topic 6:
vote bidenharris2020 bidenharris maga don
Topic 7:
donaldtrump uselection2020 elections2020 donald usaelections2020
Topic 8:
president elect vice states united
Topic 9:
election2020 electionday elections2020 bidenharris2020 electionnight
Topic 10:
die der und ist das
Topic 11:
realdonaldtrump potus maga trump2020 gop
Topic 12:
le les et est il
Topic 13:
election 2020 votes win day
Topic 14:
just like don know going
Topic 15:
america great make maga world
Topic 16:
joe donald kamala hunter harris
Topic 17:
people american don voted know
Topic 18:
la di che del se
Topic 19:
usa coronavirus covid covid19 r2p


In [330]:
final_df['NMF_Topic'] = nmf_topics.argmax(axis=1)
nmf_mapping = {0: 'donald trump maga', 1: None, 2:'biden harris 2020', 3: 'joebiden hunterbiden kamalaharris',
              4: 'spanish tweet', 5: None, 6: 'vote 2020', 7: 'donald trump maga', 8: 'president and vice president', 9: 'election 2020', 
               10: 'german tweet', 11: 'donald trump maga', 12: 'french tweet', 13: 'elections 2020', 14: None, 15: 'MAGA',
              16: 'election 2020', 17: 'joebiden hunterbiden kamalaharris', 18: None, 19: 'COVID19'} 


In [331]:
# Map the topics based on the indices in 'NMF_Topic'
final_df['Topic'] = final_df['NMF_Topic'].map(nmf_mapping)

In [332]:
final_df.drop(columns=['NMF_Topic'], inplace=True)

In [333]:
#Initialize VADER
sia = SentimentIntensityAnalyzer()

# Function to get sentiment
def get_sentiment(text):
    return sia.polarity_scores(text)['compound']  # Returns the compound score

In [334]:
from tqdm import tqdm
tqdm.pandas(desc="Analyzing Sentiments")

# Apply sentiment analysis with progress tracking
final_df['sentiment'] = final_df['tweet'].progress_apply(get_sentiment)

# Display the DataFrame with the new sentiment column
final_df[['tweet', 'sentiment']]

Analyzing Sentiments: 100%|███████████| 667255/667255 [02:42<00:00, 4116.32it/s]


,tweet,sentiment
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0000
1,"#Trump: As a student I used to hear for years,...",0.5905
2,You get a tie! And you get a tie! #Trump ‘s ra...,0.0000
3,@CLady62 Her 15 minutes were over long time ag...,-0.4912
4,@DeeviousDenise @realDonaldTrump @nypost There...,-0.2617
...,...,...
86455,Check out my op-ed on need for End Executive O...,0.0000
86456,"Yesterday, Betty &amp; I had a great time lear...",0.6249
86457,We are forever grateful for the service and sa...,0.4588
86458,Happy first day of school @CobbSchools! #CobbB...,0.6114


In [335]:
# Define a function to categorize sentiment
def categorize_sentiment(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the categorization function
final_df['sentiment_category'] = final_df['sentiment'].apply(categorize_sentiment)


In [336]:
final_df.drop(columns=['sentiment'])

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,Topic,sentiment_category
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,spanish tweet,Neutral
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,None,Positive
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,None,Neutral
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,donald trump maga,Negative
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,donald trump maga,Negative
...,...,...,...,...,...,...,...,...,...
86455,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,Neutral
86456,None,342,RepTomPrice,None,"Yesterday, Betty &amp; I had a great time lear...",None,NaN,joebiden hunterbiden kamalaharris,Positive
86457,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,Positive
86458,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,elections 2020,Positive


In [115]:
final_df[final_df['sentiment_category'] == 'Negative'].tweet.head(50).to_csv('temp.csv')

In [117]:
final_df.columns

Index(['tweet_id', 'user_id', 'created_at', 'tweet', 'likes', 'retweet_count',
       'continent', 'country', 'state', 'city', 'state_code', 'NMF_Topic',
       'Topic', 'sentiment', 'sentiment_category'],
      dtype='object')

In [118]:
final_df.drop(columns=['continent', 'country', 'state', 'city', 'state_code', 'NMF_Topic', 'sentiment'], inplace=True)

In [119]:
final_df.rename(columns={'Topic': 'topic', 'sentiment_category' : 'sentiment'}, inplace=True)

In [289]:
final_df.head()

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,Topic,sentiment,sentiment_category
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,spanish tweet,0.0000,Neutral
1,1.3165292227484303e+18,331617619.0,MediasetTgcom24,2020-10-15 00:00:01,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,coronovirus trump,0.0000,Neutral
2,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,None,0.5905,Positive
3,1.316529227471237e+18,8.28355589206057e+17,trumpytweeter,2020-10-15 00:00:02,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,covid19,0.0000,Neutral
4,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,coronovirus trump,0.0000,Neutral


In [121]:
final_df.describe()

,retweet_count
count,580795.000000
mean,2.281886
std,58.978984
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,20615.000000


In [122]:
final_df.dtypes

tweet_id          object
user_id           object
created_at        object
tweet             object
likes             object
retweet_count    float64
topic             object
sentiment         object
dtype: object

In [123]:
final_df.to_csv('ElectionTweets.csv')

In [290]:
final_df.to_csv('ElectionTweetsUpdated.csv')

In [293]:
usersdf2.to_csv('UpdatedUsers.csv')

In [337]:
final_df.to_csv('FinalTweets.csv')

In [338]:
final_df.drop(columns=['sentiment'])

,tweet_id,user_id,user_handle,created_at,tweet,likes,retweet_count,Topic,sentiment_category
0,1.316529221557252e+18,360666534.0,elsollatinonews,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,spanish tweet,Neutral
1,1.316529228091847e+18,8436472.0,snarke,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,None,Positive
2,1.3165292523014513e+18,47413798.0,Ranaabtar,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,None,Neutral
3,1.316529291052675e+18,1138416104.0,FarrisFlagg,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,donald trump maga,Negative
4,1.3165292934979625e+18,9.007610716314296e+17,sm_gulledge,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,donald trump maga,Negative
...,...,...,...,...,...,...,...,...,...
86455,None,342,RepTomPrice,None,Check out my op-ed on need for End Executive O...,None,NaN,None,Neutral
86456,None,342,RepTomPrice,None,"Yesterday, Betty &amp; I had a great time lear...",None,NaN,joebiden hunterbiden kamalaharris,Positive
86457,None,342,RepTomPrice,None,We are forever grateful for the service and sa...,None,NaN,None,Positive
86458,None,342,RepTomPrice,None,Happy first day of school @CobbSchools! #CobbB...,None,NaN,elections 2020,Positive


In [339]:
final_df.to_csv('final tweets updated.csv')